In [1]:
"""This implementation is using just a CNN to gather predicitons on the bans using hero type data"""

'This implementation is using just a CNN to gather predicitons on the bans using hero type data'

In [22]:
'''Importing necessary libraries, i know some of these are repeted but it does not affect the performance of jupyter notebook so why not.'''
import pandas as pd
import numpy as np
import datetime
from itertools import chain

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve, GridSearchCV
from keras.models import Sequential


import tensorflow as tf
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from itertools import chain

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve, GridSearchCV

from keras.utils import to_categorical
import keras.utils

from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

'''Import the necessary Libraries'''
import pandas as pd
import numpy as np
import datetime

import tensorflow as tf
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from itertools import chain

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve, GridSearchCV

from keras.utils import to_categorical
from keras import backend as K

In [23]:
'''Now we load the dataset in the following cells'''

df = pd.read_csv("dataset_without_duplicates_Bans analysis _with_types.csv")

In [24]:
matrix1 =np.array(df.loc[:,['match_id','type1','type2','type3','type4']])
matrix1.shape

(461040, 5)

In [25]:
matrix1

array([[1928898739,          1,          1,          1,          1],
       [1928932285,          1,          1,          2,          2],
       [1928903165,          1,          3,          3,          1],
       ...,
       [1829187140,          2,          1,          2,          3],
       [1829220503,          2,          3,          3,          1],
       [1829227757,          1,          1,          1,          1]],
      dtype=int64)

In [26]:
df1 = pd.read_csv("hero_names.csv")
df1.head()

,name,hero_id,localized_name,Strength,Agility,Intelligence
0,npc_dota_hero_antimage,1,Anti-Mage,0,1,0
1,npc_dota_hero_axe,2,Axe,1,0,0
2,npc_dota_hero_bane,3,Bane,0,0,1
3,npc_dota_hero_bloodseeker,4,Bloodseeker,0,1,0
4,npc_dota_hero_crystal_maiden,5,Crystal Maiden,0,0,1


In [27]:
matrix2 =np.array(df1.loc[:,['hero_id', 'Strength','Agility','Intelligence']])

In [28]:
matrix2

array([[  1,   0,   1,   0],
       [  2,   1,   0,   0],
       [  3,   0,   0,   1],
       [  4,   0,   1,   0],
       [  5,   0,   0,   1],
       [  6,   0,   1,   0],
       [  7,   1,   0,   0],
       [  8,   0,   1,   0],
       [  9,   0,   1,   0],
       [ 10,   0,   1,   0],
       [ 11,   0,   1,   0],
       [ 12,   0,   1,   0],
       [ 13,   0,   0,   1],
       [ 14,   1,   0,   0],
       [ 15,   0,   1,   0],
       [ 16,   1,   0,   0],
       [ 17,   0,   0,   1],
       [ 18,   1,   0,   0],
       [ 19,   1,   0,   0],
       [ 20,   0,   1,   1],
       [ 21,   0,   0,   1],
       [ 22,   0,   0,   1],
       [ 23,   1,   0,   0],
       [ 24,   1,   1,   1],
       [ 25,   0,   0,   1],
       [ 26,   0,   0,   1],
       [ 27,   0,   0,   1],
       [ 28,   1,   0,   0],
       [ 29,   1,   0,   0],
       [ 30,   0,   0,   1],
       [ 31,   0,   0,   1],
       [ 32,   0,   1,   0],
       [ 33,   0,   0,   1],
       [ 34,   0,   0,   1],
       [ 35,  

In [29]:
matrix1

array([[1928898739,          1,          1,          1,          1],
       [1928932285,          1,          1,          2,          2],
       [1928903165,          1,          3,          3,          1],
       ...,
       [1829187140,          2,          1,          2,          3],
       [1829220503,          2,          3,          3,          1],
       [1829227757,          1,          1,          1,          1]],
      dtype=int64)

In [37]:
'''This fucntion helps me split the data for sequential prediction task'''
def my_split(index):
    my_input = matrix1[:,1:index]
    my_output = matrix1[:,index]
    train_x = my_input[:40000,:]
    train_y = my_output[:40000]
    test_x = my_input[40000:50000,:]
    test_y = my_output[40000:50000]
    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    return my_input,my_output,train_x,train_y,test_x,test_y

In [38]:
'''Helper funcitons for evaluation'''
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

from keras import backend as K

def my_f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def my_recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def my_precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision



In [47]:
'''Defining the model with Conv1D layer, max pooling, flatten and 2 dense layers. Optimizer used is adam with loss mse '''
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
def my_model(train_x,test_x,n_steps_val,pool_size_val,kernel_size_val):
    n_steps = n_steps_val
    n_features = 1 #This is the channels as in a image
    print("Test_X shape : ", test_x.shape)
    print("Test X : ",test_x)
    train_x_to_conv = train_x.reshape((train_x.shape[0], train_x.shape[1], n_features))
    test_x_to_conv = test_x.reshape((test_x.shape[0], test_x.shape[1], n_features))
    #print(train_x_to_conv.shape)
    # define model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps, n_features),padding = 'same',use_bias=True))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu',padding='same'))
    #model.add(MaxPooling1D(pool_size=pool_size_val))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(4,activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc',my_f1,my_recall,my_precision])
    print(model.summary())
    # fit model
    return model,train_x_to_conv,test_x_to_conv, n_features


In [50]:
import random as ra
'''This code produces evaluations for all the hero bans'''
for i in range(2,5):
    if i==2:
        n_steps_val = 1
        pool_size_val = 1
        kernel_size_val =1
    elif i==3:
        n_steps_val = 2
        pool_size_val = 1
        kernel_size_val = 2
    else:
        n_steps_val = 3
        pool_size_val = 2
        kernel_size_val  = 2
    #print("I am herrr!!!!!")
    my_input,my_output,train_x,train_y,test_x,test_y = my_split(i)
    ###K.clear_session() 
    model,train_x_to_conv,test_x_to_conv, n_features = my_model(train_x,train_y,n_steps_val,pool_size_val,kernel_size_val)
    model.fit(train_x_to_conv, train_y, epochs=2, verbose=1)
    # demonstrate prediction
    #x_input = array([70, 80, 0])
    for i in range(0,7):
        small = ra.randint(0,9999)
        big = ra.randint(0,9999)
        while ((small>=big) and ((big-small)<500)):
            small = ra.randint(0,9999)
            big = ra.randint(0,9999)
        x_input = test_x[small:big,:]
        x_input = x_input.reshape((x_input.shape[0], n_steps_val, n_features))
        yhat = model.predict(x_input, verbose=0,use_multiprocessing=True)
        #####print(yhat)
        # evaluate the model
        #loss, accuracy, precision, recall = model.evaluate(test_x, test_y, verbose=1)
        score = model.evaluate(x_input, test_y[small:big], verbose=1)
        print("Score is: ",score)

Test_X shape :  (40000, 4)
Test X :  [[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_24 (Conv1D)           (None, 1, 64)             256       
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 1, 64)             12352     
_________________________________________________________________
flatten_14 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 50)                3250      
_________________________________________________________________
dense_26 (Dense)             (None, 4)                 204       
Total params: 16,062
Trainable params: 16,062
Non-trainable params: 0
________________________________________________

1250/1250 [==============================] - 2s 2ms/step - loss: 1.0777 - acc: 0.4212 - my_f1: 0.0547 - my_recall: 0.0351 - my_precision: 0.1749
Epoch 2/2
166/166 [==============================] - 0s 1ms/step - loss: 1.0958 - acc: 0.3821 - my_f1: 0.1470 - my_recall: 0.0900 - my_precision: 0.4448
Score is:  [1.0957962274551392, 0.3820585310459137, 0.14704978466033936, 0.09001003950834274, 0.4448208510875702]
59/59 [==============================] - 0s 1ms/step - loss: 1.0938 - acc: 0.3825 - my_f1: 0.1556 - my_recall: 0.0969 - my_precision: 0.4331
Score is:  [1.0937680006027222, 0.38247862458229065, 0.15558524429798126, 0.09692796319723129, 0.43307021260261536]
192/192 [==============================] - 0s 888us/step - loss: 1.1002 - acc: 0.3747 - my_f1: 0.1406 - my_recall: 0.0861 - my_precision: 0.4266
Score is:  [1.1001955270767212, 0.37473511695861816, 0.1406288594007492, 0.08612856268882751, 0.42659905552864075]
36/36 [==============================] - 0s 1ms/step - loss: 1.0716 - a

In [19]:
print(test_y)

[40 11 75 ... 73 69 75]


In [26]:
#Credits : https://machinelearningmastery.com/how-to-develop-convolutional-neural-network-models-for-time-series-forecasting/
#!pip install tensorflow
#!pip install keras
